In [3]:
import toml

config = toml.load("../configs/db.toml")
user = config["user"]
password = config["password"]
host = config["host"]
dbname = config["dbname"]

In [4]:
from sqlalchemy import create_engine

engine = create_engine(f"postgresql://{user}:{password}@{host}/{dbname}")
connection = engine.connect()

In [5]:
import pandas as pd

gender_birth_year = pd.read_sql_query(
    """
    SELECT
        subject_id,
        (CASE gender WHEN 'M' THEN 1 ELSE 0 END) AS gender,
        (anchor_year - anchor_age) AS birth_year
    FROM mimic_core.patients
    """,
    connection,
    index_col="subject_id",
)
gender_birth_year

,gender,birth_year
subject_id,,
10000048,0,2103
10002723,0,2128
10003939,1,2184
10004222,1,2161
10005325,0,2154
...,...,...
19998203,1,2103
19998350,1,2075
19999068,1,2098


In [6]:
import numpy as np

def count_month(df: pd.DataFrame) -> pd.DataFrame:
    counts_include_zero = np.bincount(df["admit_date"].dt.month, minlength=12 + 1)
    return pd.Series(
        counts_include_zero[1:], # exclude zero
        index=[
            "jan_cnt",
            "feb_cnt",
            "mar_cnt",
            "apr_cnt",
            "may_cnt",
            "june_cnt",
            "july_cnt",
            "aug_cnt",
            "sept_cnt",
            "oct_cnt",
            "nov_cnt",
            "dec_cnt",
        ]
    )

In [7]:
import sys
from pathlib import Path

sys.path.append(str(Path("..").resolve())) # enable to import utils.icd

In [8]:
from utils.icd import load_icd

psychosis_icd = load_icd("psychosis")

In [9]:
psychosis_admit_month = pd.read_sql_query(
    f"""
    SELECT
        subject_id,
        admit_date
    FROM (
        SELECT
            subject_id,
            DATE(admittime) AS admit_date,
            icd_code,
            icd_version
        FROM mimic_hosp.diagnoses_icd
        NATURAL JOIN mimic_core.admissions
        UNION ALL
        SELECT
            subject_id,
            DATE(intime) AS admit_date,
            icd_code,
            icd_version
        FROM mimic_ed.diagnosis
        NATURAL JOIN mimic_ed.edstays
    ) AS all_diags
    WHERE (icd_version = 10 AND icd_code IN {psychosis_icd.v10})
    OR (icd_version = 9 AND icd_code IN {psychosis_icd.v9})
    """,
    connection,
    index_col="subject_id",
    parse_dates=["admit_date"]
)
psychosis_admit_month

,admit_date
subject_id,
19156627,2145-06-29
12360062,2162-10-10
16942323,2112-03-31
13495642,2148-01-18
12919793,2180-12-31
...,...
12197790,2160-02-04
17693573,2130-08-08
12296616,2162-11-16


In [10]:
psychosis_admit_month_gb = psychosis_admit_month.groupby("subject_id", sort=False)

In [11]:
psychosis_month_counts = pd.concat(
    [
        (
            psychosis_admit_month_gb
            .agg(first_psychosis_admit_date=("admit_date", "min"))
            .join(gender_birth_year)
            .assign(age=lambda df: df["first_psychosis_admit_date"].dt.year - df["birth_year"]) # calc age
            .drop(columns=["first_psychosis_admit_date", "birth_year"]) # drop used
        ),
        psychosis_admit_month_gb.apply(count_month),
    ],
    axis=1,
    verify_integrity=True,
    copy=False,
)
psychosis_month_counts

,gender,age,jan_cnt,feb_cnt,mar_cnt,apr_cnt,may_cnt,june_cnt,july_cnt,aug_cnt,sept_cnt,oct_cnt,nov_cnt,dec_cnt
subject_id,,,,,,,,,,,,,,
19156627,0,36,0,0,0,0,0,1,0,0,0,0,0,0
12360062,0,52,0,0,0,0,0,1,1,0,0,1,0,0
16942323,1,54,0,0,1,0,1,0,0,0,0,0,0,0
13495642,0,38,1,0,0,0,0,0,0,0,0,0,0,0
12919793,0,56,0,0,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17727400,0,39,0,0,0,1,0,0,0,0,0,0,0,0
14214857,0,29,0,0,0,0,1,0,0,0,0,0,0,0
11057693,0,32,0,0,0,0,1,0,0,0,0,0,0,0


In [12]:
psychosis_month_counts.to_pickle("../data/month_counts_psychosis.pkl")

In [13]:
digestive_disorders_icd = load_icd("digestive_disorders")

In [14]:
digestive_disorders_admit_month = pd.read_sql_query(
    f"""
    SELECT
        subject_id,
        admit_date
    FROM (
        SELECT
            subject_id,
            DATE(admittime) AS admit_date,
            icd_code,
            icd_version
        FROM mimic_hosp.diagnoses_icd
        NATURAL JOIN mimic_core.admissions
        UNION ALL
        SELECT
            subject_id,
            DATE(intime) AS admit_date,
            icd_code,
            icd_version
        FROM mimic_ed.diagnosis
        NATURAL JOIN mimic_ed.edstays
    ) AS all_diags
    WHERE (icd_version = 10 AND icd_code IN {digestive_disorders_icd.v10})
    OR (icd_version = 9 AND icd_code IN {digestive_disorders_icd.v9})
    """,
    connection,
    index_col="subject_id",
    parse_dates=["admit_date"]
)
digestive_disorders_admit_month

,admit_date
subject_id,
12749036,2190-12-31
10113457,2173-06-04
14997223,2137-11-30
10505944,2176-10-12
10328887,2172-09-15
...,...
15540400,2190-07-11
10940102,2112-08-15
18645022,2146-11-23


In [15]:
digestive_disorders_admit_month_gb = digestive_disorders_admit_month.groupby("subject_id", sort=False)

In [16]:
digestive_disorders_month_counts = pd.concat(
    [
        (
            digestive_disorders_admit_month_gb
            .agg(first_digestive_disorders_admit_date=("admit_date", "min"))
            .join(gender_birth_year)
            .assign(age=lambda df: df["first_digestive_disorders_admit_date"].dt.year - df["birth_year"])
            .drop(columns=["first_digestive_disorders_admit_date", "birth_year"])
        ),
        digestive_disorders_admit_month_gb.apply(count_month),
    ],
    axis=1,
    verify_integrity=True,
    copy=False,
)
digestive_disorders_month_counts

,gender,age,jan_cnt,feb_cnt,mar_cnt,apr_cnt,may_cnt,june_cnt,july_cnt,aug_cnt,sept_cnt,oct_cnt,nov_cnt,dec_cnt
subject_id,,,,,,,,,,,,,,
12749036,1,85,1,0,0,3,1,0,0,0,0,0,0,4
10113457,0,66,0,0,0,1,0,2,0,0,0,0,0,0
14997223,1,44,0,9,9,22,17,18,16,3,6,1,6,12
10505944,0,91,0,0,0,0,0,0,0,0,0,4,0,0
10328887,1,25,0,0,0,0,0,0,0,1,3,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17210052,1,71,0,0,0,0,0,0,0,0,0,0,0,1
12445708,0,45,0,0,1,0,0,0,0,0,0,0,0,0
17994489,0,52,0,0,0,0,0,0,0,0,0,0,0,1


In [17]:
digestive_disorders_month_counts.to_pickle("../data/month_counts_digestive_disorders.pkl")